In [2]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda,GRU,Embedding
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.layers import Merge


Using TensorFlow backend.


# Data

In [3]:
input1=pd.read_csv('/home/nancy/Downloads/inputque1')
input2=pd.read_csv('/home/nancy/Downloads/inputque2')
input1=input1.drop(input1.columns[0:1],axis=1)
input2=input2.drop(input2.columns[0:1],axis=1)

In [4]:
input1=input1.as_matrix()
input2=input2.as_matrix()


In [5]:
output=pd.read_csv('/home/nancy/Downloads/newdata.csv',encoding='ISO-8859-1')

In [6]:
output=output['is_duplicate']

In [7]:
output=to_categorical(output,nb_classes=2)

In [8]:
len(output)

298860

# Train Validate Test = 0.8 : 0.1 : 0.1

In [9]:
inputtrain2=input2[0:int(0.8*len(input1))]
inputvalid2=input2[int(0.8*len(input1)):int(0.9*len(input1))]
inputtest2=input2[int(0.9*len(input1)):]

In [10]:
inputtrain1=input1[0:int(0.8*len(input1))]
inputvalid1=input1[int(0.8*len(input1)):int(0.9*len(input1))]
inputtest1=input1[int(0.9*len(input1)):]

In [11]:
outputtrain=output[0:int(0.8*len(input1))]
outputval=output[int(0.8*len(input1)):int(0.9*len(input1))]
outputtest=output[int(0.9*len(input1)):]

# Loading of Pre-trained word-vector

In [12]:
word_embedding=pd.read_csv('/home/nancy/Downloads/glove.6B/vocab2.csv')


In [13]:
word_embedding=word_embedding.drop(word_embedding.columns[0:1],axis=1)
#id's

In [14]:
word_embedding=word_embedding.as_matrix()

# Model Architecture 

## Model_1

In [15]:
model1=Sequential()
model1.add(Embedding(word_embedding.shape[0],word_embedding.shape[1],input_length=50,mask_zero=True,trainable=False,weights=[word_embedding]))
model1.add(GRU(output_dim=128,return_sequences=False,go_backwards=True,init='he_normal'))
model1.add(Dropout(0.5))
#model1.add(Dense(1,activation='tanh'))
#model1.compile(loss='mse',optimizer='rmsprop',metrics=['accuracy'])
#model1.fit(inputk2,outputk,nb_epoch=10,batch_size=128)

In [16]:
model1.layers[0].output_shape

(None, 50, 100)

## Model_2

In [17]:
model2=Sequential()
model2.add(Embedding(word_embedding.shape[0],word_embedding.shape[1],input_length=50,mask_zero=True,trainable=False,weights=[word_embedding]))
model2.add(GRU(output_dim=128,return_sequences=False,go_backwards=True,init='he_normal'))
model2.add(Dropout(0.5))

## Merging Of Models 1 and 2

In [18]:
main_model=Sequential()
merged=Merge([model1,model2],mode=lambda x: ((x[0]-x[1])*(x[0]-x[1])),output_shape=[128])
main_model.add(merged)
main_model.add(Dense(2,activation='softmax'))


In [19]:
main_model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy'])


# Checkpoints for saving model where val-acc improve

In [20]:
filepath='/home/nancy/Downloads/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

# Training

In [21]:
main_model.fit([inputtrain1,inputtrain2],outputtrain,nb_epoch=1, batch_size=128,verbose=2,shuffle=True,callbacks=callbacks_list,validation_data=([inputvalid1,inputvalid2],outputval))

Train on 239088 samples, validate on 29886 samples
Epoch 1/1
Epoch 00000: val_acc improved from -inf to 0.74165, saving model to /home/nancy/Downloads/weights-improvement-00-0.74.hdf5
1387s - loss: 0.1963 - acc: 0.7027 - val_loss: 0.1870 - val_acc: 0.7417


In [19]:
main_model.load_weights('/home/nancy/Desktop/mainweight.h5')

# Evaluate

In [22]:
score = main_model.evaluate([inputtest1,inputtest2], outputtest, batch_size=128)

29886/29886 [==============================] - 72s     

In [26]:
#loss,accuracy
score

[0.18727663118784377, 0.74027972965525124]